### Connect to Gremlin Server

In [2]:
import os
import sys
from pathlib import Path

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)


from gremlin_python import statics
from gremlin_python.process.traversal import T, Direction
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import GraphTraversalSource
from gremlin_python.process.graph_traversal import __

from graph.base import g
from ipycytoscape_graph_visualization import visualize_graph

from dotenv import load_dotenv

load_dotenv()

from concurrent.futures import ThreadPoolExecutor
from contextlib import contextmanager
from queue import Queue

# Queue to hold statements
statement_queue = Queue()

# Function to add statement to queue
def add_to_queue(statement):
    statement_queue.put(statement)

@contextmanager
def execute_gremlin_statements():
    try:
        yield add_to_queue
    finally:
        statements = []
        while not statement_queue.empty():
            statements.append(statement_queue.get())
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(statement) for statement in statements]
            results = [future.result() for future in futures]

        for result in results:
            print(result)

# test connection to gremlin server
with execute_gremlin_statements() as execute:
    execute(lambda: g.V().limit(1).toList())

[]


### Get Networkx Graph

In [1]:
import networkx as nx

from models.base import SessionLocal
from models.bitcoin_data import Block, Tx, Address, Input, Output
from graph.base import g
from graph_analyze import GraphAnalyzer


analyzer = GraphAnalyzer(g, SessionLocal)

interesting_addr = '12higDjoCCNXSA95xZMWUdPvXNmkAduhWv'

with SessionLocal() as session:
    address = session.query(Address).filter_by(addr=interesting_addr).first()

print(f"id of address {address.addr:4}: {address.id}")


my_hist = None
graph = None

def hist():
    global my_hist, graph, analyzer
    my_hist = analyzer.get_address_history(interesting_addr, 'address')
    graph = analyzer.to_networkx(my_hist)

with execute_gremlin_statements() as add_statement:
    add_statement(lambda: hist())

print(my_hist)
print(graph)

# dump graph to gexf file
nx.write_gexf(graph, "addr_hist_graph.gexf")

ModuleNotFoundError: No module named 'models'

In [6]:
print(my_hist)

[{<T.id: 1>: 23503008, <T.label: 4>: 'output', 'id': 556}, {<T.id: 1>: 23589072, <T.label: 4>: 'output', 'id': 419}, {<T.id: 1>: 23609504, <T.label: 4>: 'output', 'id': 693}, {<T.id: 1>: 23957584, <T.label: 4>: 'output', 'id': 759}, {<T.id: 1>: 23961808, <T.label: 4>: 'output', 'id': 843}, {<T.id: 1>: 71020720, <T.label: 4>: 'output', 'id': 620}, {<T.id: 1>: 23629904, <T.label: 4>: 'output', 'id': 341}, {<T.id: 1>: 24248480, <T.label: 4>: 'output', 'id': 1426}, {<T.id: 1>: 24256720, <T.label: 4>: 'output', 'id': 1151}, {<T.id: 1>: 24268960, <T.label: 4>: 'output', 'id': 1434}, {<T.id: 1>: 24273144, <T.label: 4>: 'output', 'id': 1377}, {<T.id: 1>: 24289440, <T.label: 4>: 'address', 'id': 1432}, {<T.id: 1>: 24305872, <T.label: 4>: 'address', 'id': 1176}, {<T.id: 1>: 24326392, <T.label: 4>: 'output', 'id': 1435}, {<T.id: 1>: 24346872, <T.label: 4>: 'address', 'id': 1433}, {<T.id: 1>: 24428792, <T.label: 4>: 'output', 'id': 1557}, {<T.id: 1>: 24596728, <T.label: 4>: 'address', 'id': 1781},

### Find Coin Sources

In [2]:
def get_sources(g, node_name):
    source_record = {}
    def traverse_sources(vertex, fraction=1.0):
        # Traverse incoming edges
        in_edges = g.V(vertex).inE('sent').toList()
        for edge in in_edges:
            sender = edge.outV().next()
            amount_from_sender = edge['value']
            if 'marked' in sender.properties:
                # Add record if sender is marked
                if sender['name'] in source_record:
                    source_record[sender['name']] += amount_from_sender * fraction
                else:
                    source_record[sender['name']] = amount_from_sender * fraction
            # Recursive case
            if g.V(sender).inE('sent').hasNext():
                sender_total_received = sum(e['value'] for e in g.V(sender).inE('sent').toList())
                amount_fraction = (amount_from_sender / sender_total_received) * fraction
                traverse_sources(sender, amount_fraction)

    # Start traversal
    traverse_sources(g.V().has('name', node_name).next())
    return source_record

print("Sources for Bob:")
print(get_sources(g, 'Bob'))

Sources for Bob:


RuntimeError: Cannot run the event loop while another loop is running

### Execute Some Queries

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# def add_vertex():
#     return g.addV('Address').property('addr', 'some_address').next()

# with ThreadPoolExecutor() as executor:
#     future = executor.submit(add_vertex)
#     result = future.result()
    # print(result)
    
with execute_gremlin_statements() as add_statement:
    add_statement(lambda: g.addV('Address').property('addr', 'some_address').next())

v[4288]
